In [ ]:
import sys
sys.path.append('/Users/karl/mrl/')

In [ ]:
from mrl.imports import *
from mrl.core import *
from mrl.chem import *
from mrl.templates import *

//anaconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *
from mrl.dataloaders import *
from mrl.g_models import *
from mrl.agent import *

In [ ]:
values = torch.rand((32, 90))

In [ ]:
mask = torch.ones(values.shape)
mask[:, 75:] = 0
mask = mask.bool()

In [ ]:
mask

tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]])

In [ ]:
values

tensor([[0.7341, 0.1199, 0.7577,  ..., 0.3650, 0.2875, 0.1340],
        [0.1224, 0.0810, 0.0088,  ..., 0.1404, 0.8838, 0.7014],
        [0.1240, 0.1228, 0.1735,  ..., 0.6357, 0.5716, 0.8272],
        ...,
        [0.0856, 0.8900, 0.5229,  ..., 0.4426, 0.8995, 0.6533],
        [0.2605, 0.3823, 0.8008,  ..., 0.5164, 0.8860, 0.3829],
        [0.2991, 0.2311, 0.8919,  ..., 0.3108, 0.4898, 0.6336]])

In [ ]:
whiten(values)

tensor([[ 0.8125, -1.3386,  0.8953,  ..., -0.4803, -0.7517, -1.2892],
        [-1.3297, -1.4749, -1.7278,  ..., -1.2667,  1.3369,  0.6980],
        [-1.3244, -1.3284, -1.1509,  ...,  0.4678,  0.2433,  1.1388],
        ...,
        [-1.4587,  1.3587,  0.0729,  ..., -0.2084,  1.3918,  0.5296],
        [-0.8462, -0.4195,  1.0463,  ...,  0.0501,  1.3445, -0.4174],
        [-0.7110, -0.9490,  1.3654,  ..., -0.6700, -0.0429,  0.4606]])

In [ ]:
(values - values.mean()).pow(2).sum()/(values.numel()-1)-values.var()

tensor(0.)

In [ ]:
values.var()

tensor(0.0815)

In [ ]:
values.mean()

tensor(0.5021)

In [ ]:
values.sum()/values.numel()

tensor(0.5021)

In [ ]:
(values*mask).sum()/mask.sum()

tensor(0.5024)

In [ ]:
values.numel()

2880

In [ ]:
def whiten2(values, shift_mean=True, mask=None):
    if mask is None:
        mean = values.mean()
        var = values.var()
    else:
        mean = (values*mask).sum()/mask.sum()
        var = ((values-mean)*mask).pow(2).sum()/(mask.sum()-1)
        
    whitened = (values - mean) * torch.rsqrt(var + 1e-8)
    
    if not shift_mean:
        whitened += mean
        
    if mask is not None:
        whitened = whitened*mask
        
    return whitened

In [ ]:
whiten2(values)

tensor([[ 0.8125, -1.3386,  0.8953,  ..., -0.4803, -0.7517, -1.2892],
        [-1.3297, -1.4749, -1.7278,  ..., -1.2667,  1.3369,  0.6980],
        [-1.3244, -1.3284, -1.1509,  ...,  0.4678,  0.2433,  1.1388],
        ...,
        [-1.4587,  1.3587,  0.0729,  ..., -0.2084,  1.3918,  0.5296],
        [-0.8462, -0.4195,  1.0463,  ...,  0.0501,  1.3445, -0.4174],
        [-0.7110, -0.9490,  1.3654,  ..., -0.6700, -0.0429,  0.4606]])

In [ ]:
whiten2(values, mask=torch.ones(values.shape))

tensor([[ 0.8125, -1.3386,  0.8953,  ..., -0.4803, -0.7517, -1.2892],
        [-1.3297, -1.4749, -1.7278,  ..., -1.2667,  1.3369,  0.6980],
        [-1.3244, -1.3284, -1.1509,  ...,  0.4678,  0.2433,  1.1388],
        ...,
        [-1.4587,  1.3587,  0.0729,  ..., -0.2084,  1.3918,  0.5296],
        [-0.8462, -0.4195,  1.0463,  ...,  0.0501,  1.3445, -0.4174],
        [-0.7110, -0.9490,  1.3654,  ..., -0.6700, -0.0429,  0.4606]])

In [ ]:
(whiten2(values)==whiten2(values, mask=torch.ones(values.shape))).all()

tensor(True)

In [ ]:
whiten2(values, mask=mask)

tensor([[ 0.8113, -1.3390,  0.8941,  ..., -0.0000, -0.0000, -0.0000],
        [-1.3302, -1.4754, -1.7282,  ..., -0.0000,  0.0000,  0.0000],
        [-1.3248, -1.3289, -1.1514,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.4592,  1.3574,  0.0720,  ..., -0.0000,  0.0000,  0.0000],
        [-0.8468, -0.4203,  1.0450,  ...,  0.0000,  0.0000, -0.0000],
        [-0.7116, -0.9496,  1.3641,  ..., -0.0000, -0.0000,  0.0000]])

In [ ]:
    mean, var = torch.mean(values), torch.var(values)
    whitened = (values - mean) * torch.rsqrt(var + 1e-8)
    if not shift_mean:
        whitened += mean
    return whitened

tensor([[0.7341, 0.1199, 0.7577,  ..., 0.3650, 0.2875, 0.1340],
        [0.1224, 0.0810, 0.0088,  ..., 0.1404, 0.8838, 0.7014],
        [0.1240, 0.1228, 0.1735,  ..., 0.6357, 0.5716, 0.8272],
        ...,
        [0.0856, 0.8900, 0.5229,  ..., 0.4426, 0.8995, 0.6533],
        [0.2605, 0.3823, 0.8008,  ..., 0.5164, 0.8860, 0.3829],
        [0.2991, 0.2311, 0.8919,  ..., 0.3108, 0.4898, 0.6336]])

In [ ]:
class PoolingHead(nn.Module):
    def __init__(self, d_in, dims, d_out, drops, outrange=None):
        super().__init__()
        
        self.layers = MLP_Encoder(d_in, dims, d_out, drops)
        self.outrange = outrange
        
    def forward(self, x, mask=None):
        # x - bs, sl, d
        # mask - bs, sl
        if mask is not None:
            lengths = mask.sum(-1)
            final_vals = x[torch.arange(x.shape[0]), lengths-1]
            pool1 = x.masked_fill(mask.unsqueeze(-1), 0).sum(1)/lengths.unsqueeze(-1)
            pool2 = x.masked_fill(mask.unsqueeze(-1), -float('inf')).max(1)[0]
        else:
            final_vals = x[:,-1]
            pool1 = x.mean(1)
            pool2 = x.max(1)[0]
            
        x = torch.cat([final_vals, pool1, pool2], 1)
        x = self.layers(x)
        
        if self.outrange is not None:
            x = torch.sigmoid(x) * (self.outrange[1]-self.outrange[0]) + self.outrange[0]
        
        return x
    
class Predictive_LSTM(nn.Module):
    def __init__(self, d_vocab, d_embedding, d_hidden, n_layers, d_out_lstm,
                 head_dims, head_drops, d_out, outrange=None,
                 lstm_drop=0., bidir=False,):
        super().__init__()
        
        self.embedding = nn.Embedding(d_vocab, d_embedding)
        self.lstm = LSTM(d_embedding, d_hidden, d_out_lstm, n_layers,
                                     bidir=bidir, dropout=lstm_drop)
        
        self.head = PoolingHead(d_out_lstm*3, head_dims, d_out, head_drops, outrange)

        
    def forward(self, x, hiddens=None, mask=None):
        
        x = self.embedding(x)
        encoded, hiddens = self.lstm(x, hiddens)
        output = self.head(encoded, mask)
        return output
    
    def freeze_encoder(self):
        for p in self.embedding.parameters():
            p.requires_grad_(False)
            
        for p in self.lstm.parameters():
            p.requires_grad_(False)
    
    def load_from_lm(self, lm_model):
        if hasattr(lm_model, 'block'):
            self.embedding.load_state_dict(lm_model.block.embedding.state_dict())
            self.lstm.load_state_dict(lm_model.block.lstm.state_dict())
        else:
            if hasattr(lm_model, 'lstm'):
                self.lstm.load_state_dict(lm_model.lstm.state_dict())
                
            if hasattr(lm_model, 'embedding'):
                self.embedding.load_state_dict(lm_model.embedding.state_dict())

In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
ds = TextDataset(['CCC'], vocab)

d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
lstm_drop = 0.
lin_drop = 0.
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

lm_model = LSTM_LM(d_vocab, d_embedding, d_hidden, n_layers,
                lstm_drop, lin_drop, bos_idx, bidir, tie_weights)

lm_model.load_state_dict(torch.load('../nbs/untracked_files/lstm_lm_small.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model = Predictive_LSTM(d_vocab, d_embedding, d_hidden, n_layers, d_embedding,
                        [512, 256], [0.1, 0.1], 1)

In [ ]:
model.load_from_lm(lm_model)

In [ ]:
next(model.lstm.parameters())

Parameter containing:
tensor([[ 0.0181,  0.0324,  0.0202,  ...,  0.0010,  0.0047,  0.0283],
        [-0.0101,  0.0228,  0.0193,  ..., -0.0366,  0.0038, -0.0540],
        [ 0.0537, -0.0437, -0.0073,  ...,  0.0270, -0.0121,  0.0504],
        ...,
        [ 0.0567,  0.0338, -0.0113,  ..., -0.0163,  0.0104,  0.0140],
        [ 0.0025,  0.0101,  0.0410,  ..., -0.0130, -0.0164, -0.0144],
        [ 0.0174,  0.0205,  0.0073,  ...,  0.0442, -0.0021,  0.0154]],
       requires_grad=True)

In [ ]:
df = pd.read_csv('../nbs/files/smiles.csv')

In [ ]:
def masked_sequence_prediction_collate(batch, pad_idx, batch_first=True):
    
    x,y = sequence_prediction_collate(batch, pad_idx, batch_first)
    mask = ~(x==pad_idx)
    return ((x,None,mask), y)

In [ ]:
collate_fn = partial(masked_sequence_prediction_collate, pad_idx=vocab.stoi['pad'])

In [ ]:
ds = TextPredictionDataset(df.smiles.values, np.array([0.]*df.shape[0]), vocab,
                          collate_function=collate_fn)

In [ ]:
ds[0]

(tensor([ 0, 22, 26, 33, 11, 36, 33,  5, 29, 22, 22,  5, 19, 27,  6, 26, 33, 12,
         33, 33,  5, 22, 35,  6, 33, 33, 33, 12, 27, 22,  6, 36, 33, 12, 33, 33,
         33, 33, 33, 11, 12,  1]), tensor([0.]))

In [ ]:
x,y = next(iter(ds.dataloader(16)))

In [ ]:
x

(tensor([[ 0, 22, 26,  ...,  2,  2,  2],
         [ 0, 22, 27,  ...,  2,  2,  2],
         [ 0, 22, 33,  ...,  2,  2,  2],
         ...,
         [ 0, 22, 33,  ...,  2,  2,  2],
         [ 0, 22, 22,  ...,  2,  2,  2],
         [ 0, 27, 19,  ...,  2,  2,  2]]),
 None,
 tensor([[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False]]))

In [ ]:
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
with torch.no_grad():
    out = model(*x)

In [ ]:
out

tensor([[-0.2800],
        [ 0.0608],
        [-0.1827],
        [-0.2925],
        [ 0.2265],
        [ 0.2253],
        [ 0.0656],
        [-0.0847],
        [-0.8578],
        [-0.3824],
        [-0.1353],
        [-0.4450],
        [ 0.0713],
        [ 0.5608],
        [ 0.3842],
        [-0.3874]])

In [ ]:
0.99907**1000

0.3943830166921718

In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
ds = TextDataset(['CCC'], vocab)

d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
lstm_drop = 0.
lin_drop = 0.
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

model = LSTM_LM(d_vocab, d_embedding, d_hidden, n_layers,
                lstm_drop, lin_drop, bos_idx, bidir, tie_weights)

model.load_state_dict(torch.load('../nbs/untracked_files/lstm_lm_small.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
hasattr(model, 'block')

True

In [ ]:
model

LSTM_LM(
  (block): LSTM_Block(
    (embedding): Embedding(42, 256)
    (lstm): LSTM(
      (lstms): ModuleList(
        (0): LSTM(256, 1024, batch_first=True)
        (1): LSTM(1024, 1024, batch_first=True)
        (2): LSTM(1024, 256, batch_first=True)
      )
    )
    (head): Linear(in_features=256, out_features=42, bias=True)
    (head_drop): Dropout(p=0.0, inplace=False)
  )
)

In [ ]:
class ValueHead(nn.Module):
    def __init__(self, d_in, dropout=0.):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        self.layer = nn.Linear(d_in, 1)
        
    def forward(self, x):
        x = self.layer(self.drop(x)).squeeze(-1)
        return x

In [ ]:
vh = ValueHead(256)
# vh = None

In [ ]:
agent = GenerativeAgent(model, vocab, CrossEntropy(), ds, value_head=vh)

In [ ]:
next(agent.model.parameters())

Parameter containing:
tensor([[-0.1016, -0.0330,  0.0345,  ..., -0.0337,  0.1236,  0.0771],
        [ 0.4183, -0.0701, -0.2043,  ...,  0.5718,  0.0684,  0.4263],
        [ 0.0120, -0.1297,  0.2431,  ...,  0.4713,  0.0100, -0.1059],
        ...,
        [ 0.2554,  0.3921, -0.3530,  ...,  0.4462,  0.2011,  0.5215],
        [-0.0203, -0.0836, -0.0156,  ...,  0.0513,  0.0746,  0.0285],
        [-0.0364, -0.0647,  0.0147,  ...,  0.0118,  0.0850, -0.0063]],
       requires_grad=True)

In [ ]:
next(agent.base_model.parameters())

Parameter containing:
tensor([[-0.1016, -0.0330,  0.0345,  ..., -0.0337,  0.1236,  0.0771],
        [ 0.4183, -0.0701, -0.2043,  ...,  0.5718,  0.0684,  0.4263],
        [ 0.0120, -0.1297,  0.2431,  ...,  0.4713,  0.0100, -0.1059],
        ...,
        [ 0.2554,  0.3921, -0.3530,  ...,  0.4462,  0.2011,  0.5215],
        [-0.0203, -0.0836, -0.0156,  ...,  0.0513,  0.0746,  0.0285],
        [-0.0364, -0.0647,  0.0147,  ...,  0.0118,  0.0850, -0.0063]],
       requires_grad=True)

In [ ]:
next(agent.value_head.parameters())

Parameter containing:
tensor([[ 0.0140,  0.0069, -0.0449,  0.0479, -0.0507,  0.0434, -0.0438,  0.0026,
          0.0118,  0.0359,  0.0568, -0.0576,  0.0605,  0.0242, -0.0431,  0.0614,
         -0.0378, -0.0133,  0.0037,  0.0421, -0.0426, -0.0187,  0.0436,  0.0538,
          0.0446,  0.0256, -0.0123, -0.0343, -0.0336,  0.0550, -0.0334, -0.0427,
          0.0075,  0.0466, -0.0471,  0.0495,  0.0090,  0.0018,  0.0119,  0.0143,
          0.0589,  0.0125, -0.0606, -0.0584,  0.0426,  0.0563, -0.0590,  0.0511,
         -0.0495,  0.0025,  0.0070,  0.0402, -0.0433,  0.0615, -0.0368, -0.0112,
         -0.0046, -0.0002,  0.0043, -0.0556,  0.0393, -0.0159,  0.0137,  0.0244,
         -0.0609, -0.0475,  0.0141, -0.0191, -0.0018,  0.0452, -0.0250,  0.0153,
         -0.0199, -0.0003,  0.0340,  0.0619, -0.0260, -0.0436,  0.0372,  0.0199,
          0.0144,  0.0066,  0.0513, -0.0189, -0.0090,  0.0267,  0.0505, -0.0304,
          0.0537,  0.0531, -0.0265, -0.0515,  0.0482,  0.0437,  0.0426, -0.0016,
      

In [ ]:
preds, _ = agent.model.sample_no_grad(32, 80)
smiles = agent.reconstruct(preds)
smiles = [i for i in smiles if to_mol(i) is not None]

In [ ]:
buffer_ds = agent.dataset.new(smiles)
buffer_dl = buffer_ds.dataloader(16, shuffle=True)

In [ ]:
x,y = next(iter(buffer_dl))

In [ ]:
model_output = ModelOutput()

In [ ]:
pad_idx = vocab.stoi['pad']

mask = ~(y==pad_idx)
lengths = mask.sum(-1)
sl = y.shape[-1]

model_output['x'] = x
model_output['y'] = y
model_output['mask'] = mask
model_output['lengths'] = lengths
model_output['sl'] = sl

In [ ]:
model_output = agent.get_model_outputs(model_output)

In [ ]:
model_output['state_values'].shape

torch.Size([16, 69])

In [ ]:
model_output['sequences'] = agent.reconstruct(x)
model_output['mols'] = to_mols(model_output['sequences'])
model_output['source'] = ['model']*x.shape[0]

In [ ]:
x.shape

torch.Size([16, 69])

In [ ]:
rewards = torch.tensor([qed(i) for i in model_output['mols']])
rewards_scaled = rewards - rewards.mean()

model_output['rewards'] = rewards
model_output['rewards_scaled'] = rewards_scaled

In [ ]:
from mrl.policy_gradient import PolicyGradient, TRPO, PPO

In [ ]:
# pg = PolicyGradient(discount=False)
# pg = PolicyGradient(discount=True)
# pg = TRPO(0.98, 2)
pg = PPO(0.98, 0.2)

In [ ]:
model_output = pg(model_output)

In [ ]:
model_output['pg_loss'].shape

torch.Size([])

In [ ]:
model_output['pg_loss']

tensor(0.0931, grad_fn=<SubBackward0>)

In [ ]:
agent.zero_grad()

In [ ]:
loss = model_output['pg_loss']

In [ ]:
loss.backward()

In [ ]:
agent.step()

In [ ]:
next(agent.model.parameters())

Parameter containing:
tensor([[-0.1026, -0.0340,  0.0355,  ..., -0.0347,  0.1226,  0.0761],
        [ 0.4193, -0.0691, -0.2033,  ...,  0.5708,  0.0674,  0.4253],
        [ 0.0130, -0.1287,  0.2441,  ...,  0.4723,  0.0090, -0.1069],
        ...,
        [ 0.2544,  0.3911, -0.3540,  ...,  0.4472,  0.2001,  0.5225],
        [-0.0196, -0.0838, -0.0153,  ...,  0.0516,  0.0745,  0.0281],
        [-0.0357, -0.0649,  0.0151,  ...,  0.0120,  0.0849, -0.0067]],
       requires_grad=True)

In [ ]:
next(agent.base_model.parameters())

Parameter containing:
tensor([[-0.1016, -0.0330,  0.0345,  ..., -0.0337,  0.1236,  0.0771],
        [ 0.4183, -0.0701, -0.2043,  ...,  0.5718,  0.0684,  0.4263],
        [ 0.0120, -0.1297,  0.2431,  ...,  0.4713,  0.0100, -0.1059],
        ...,
        [ 0.2554,  0.3921, -0.3530,  ...,  0.4462,  0.2011,  0.5215],
        [-0.0203, -0.0836, -0.0156,  ...,  0.0513,  0.0746,  0.0285],
        [-0.0364, -0.0647,  0.0147,  ...,  0.0118,  0.0850, -0.0063]],
       requires_grad=True)

In [ ]:
next(agent.value_head.parameters())

Parameter containing:
tensor([[ 0.0130,  0.0059, -0.0439,  0.0469, -0.0497,  0.0444, -0.0428,  0.0036,
          0.0128,  0.0349,  0.0578, -0.0566,  0.0595,  0.0252, -0.0421,  0.0624,
         -0.0368, -0.0123,  0.0047,  0.0431, -0.0416, -0.0197,  0.0426,  0.0528,
          0.0456,  0.0246, -0.0133, -0.0333, -0.0346,  0.0540, -0.0324, -0.0417,
          0.0085,  0.0476, -0.0481,  0.0485,  0.0100,  0.0008,  0.0109,  0.0133,
          0.0599,  0.0115, -0.0616, -0.0574,  0.0416,  0.0553, -0.0580,  0.0521,
         -0.0485,  0.0035,  0.0080,  0.0392, -0.0423,  0.0605, -0.0358, -0.0102,
         -0.0036,  0.0008,  0.0053, -0.0566,  0.0383, -0.0169,  0.0147,  0.0234,
         -0.0599, -0.0465,  0.0151, -0.0181, -0.0028,  0.0462, -0.0240,  0.0143,
         -0.0189, -0.0013,  0.0330,  0.0629, -0.0250, -0.0446,  0.0362,  0.0209,
          0.0134,  0.0076,  0.0523, -0.0179, -0.0080,  0.0277,  0.0495, -0.0314,
          0.0547,  0.0521, -0.0255, -0.0525,  0.0492,  0.0427,  0.0436, -0.0026,
      

In [ ]:
model_output.keys()

dict_keys(['sequences', 'mols', 'source', 'x', 'y', 'mask', 'lengths', 'sl', 'model_output', 'model_encoded', 'model_logprobs', 'model_gathered_logprobs', 'state_values', 'reference_output', 'reference_encoded', 'reference_logprobs', 'reference_gathered_logprobs', 'rewards', 'rewards_dict', 'rewards_scaled', 'trajectory_rewards', 'diff_loss', 'diff_loss_dict', 'pg_loss', 'pg_dict'])

In [ ]:
model_output['pg_dict']

{'pg_discounted': tensor([[-0.1271, -0.1297, -0.1324,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0212, -0.0216, -0.0221,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0297,  0.0303,  0.0310,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0823,  0.0840,  0.0857,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0858,  0.0876,  0.0894,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0336, -0.0343, -0.0350,  ...,  0.0000,  0.0000,  0.0000]]),
 'pg_advantage': tensor([[-1.9058, -2.2628, -2.3443,  ..., -0.1343, -0.1625, -0.1959],
         [ 0.4370,  0.1088, -0.3839,  ..., -0.2189, -0.2605, -0.3029],
         [ 1.4972,  1.1780,  0.8214,  ..., -0.0632, -0.0540, -0.0557],
         ...,
         [ 2.5539,  2.2412,  2.1674,  ..., -0.1722, -0.2109, -0.2519],
         [ 2.6634,  2.3540,  2.0062,  ..., -0.2369, -0.2802, -0.3318],
         [ 0.1938, -0.1355, -0.5012,  ...,  0.0108, -0.0298, -0.0798]]),
 'ratios': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 

In [ ]:
model_output['model_encoded'].shape

torch.Size([16, 72, 256])

In [ ]:
class ValueHead(nn.module):
    def __init__(self, d_in, dropout=0.):
        self.drop = nn.Dropout(dropout)
        self.layer = nn.Linear(d_in, 1)
        
    def forward(self, x):
        x = self.layer(self.drop(x))
        return x

In [ ]:
class SpecialDict(dict):
    def __init__(self):
        super().__init__()
        
    def __setitem__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
    
    def __setattr__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
        
#     def __setitem__(self, key, item):
#         super().__setitem__(key, item)
#         self.__setattr__(key, item)
    
#     def __setattr_(self, key, item):
#         super().__setattr__(key, item)
#         self.__setitem__(key, item)

In [ ]:
d = SpecialDict()

In [ ]:
d['_5'] = 2

In [ ]:
d

{'_5': 2}

In [ ]:
d._5

2

In [ ]:
d['event'] = 'test'

In [ ]:
d.event

'test'

In [ ]:
d

{'_5': 2, 'event': 'test'}

In [ ]:
for (k,v) in d.items():
    print(k,v)

_5 2
event test


In [ ]:
d.rest = 5

In [ ]:
d.rest

5

In [ ]:
d

{'_5': 2, 'event': 'test', 'rest': 5}

In [ ]:
class Env():
    
    def fit(self, bs, buffer_size,):
        self.bs = bs
        self.buffer_size = buffer_size
        self.iterations
        self.build_buffer() # automatically as before batch?
        

before rewrite
    get more rewards working
        batch vs paarallel
        logging in rewaard plus lookup 
            index grab
    get contrastive working
    compute rewards in bulk from buffer
    diff loss on learned latents
    break out samples
    timers during fit loop
    protein LM
    


current roadmap
    get contrastive working
    write better reward function module
        logging
        parallel processing
    start basic protein LM
    start big rewrite
        everything from torch core on gets rewritten
        scrap encoder/decoder class
        make VAE its own thing (untangle conditional lstm lm)
        think about handling inputs to vae vs conditional lstm
        after layers, go straight to callbacks/environment
            keep things lean
        merge double agent thing we have going on
        do reward functions
        revisit mol/protein distinctions in template/block
    now we do documentation/update site
    combichem
    selies
    jtnnvae
    start working on other docs
        guides
        examples
    move to release v0
    
    

future modules
    combichem
    selfies support
    pharmacophore
    new models
        jtnnvae
        flow based
    chemprop hooks
    huggingface hooks
        
project quality
    collab links
    google search on site
    more documentation

Use cases/examples
    basic drug design
    iterative drug design with dataset building
    drug design with docking proxy
    proteins/antibodies
    polymers
    catalyst
    materials
    
guides
    generative screening primer
    basics
        stuff about how bad score functions can be
    how to use callbacks
    basic drug design
    contrastive optimization
    latent/prior optimization
    training tips/tricks (callback stuff)
    
paapers to implement
    fastai guy - conditional lstm, contrastive
    moldqn - dqn sampler (rollout), agent is score function, dqn loss
    synthesis constrained? really same as DQN with different rolllout
    stoned selfies/genetic algorithm examples